In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import dfm

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo


In [4]:
def gen_pattern_replace_and_matcher_for_SingleMLP(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])


      def forward(self,x):
          x = self.embed(x)
          square_of_sum = torch.sum(x, dim=1) ** 2
          sum_of_square = torch.sum(x ** 2, dim=1)
          ix = square_of_sum - sum_of_square    
          ix = torch.sum(ix, dim=1, keepdim=True)  
          return self.mlp(x.view(-1,self.embed_output_dim)), 0.5 * ix
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)

      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)

      

    def forward(self,x):
      emb = self.embed(x)
      square_of_sum = torch.sum(emb, dim=1) ** 2
      sum_of_square = torch.sum(emb ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      redency_part = emb[redency_part_slice] 
      unredency_part = emb[unredency_part_slice] 
      return self.redency_linear(redency_part.view(-1,self.redency_weight_len)) + self.unredency_linear(unredency_part.view(-1,self.unredency_weight_len)), 0.5 * ix
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [5]:
def workload_dfm(num_field, prefix,dim = 64,l = [1024,512,256]):
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  DFM_model_ori = dfm.DeepFactorizationMachineModel([100 for i in range(num_field)],dim,l,0.1)
  ori_traced = symbolic_trace(DFM_model_ori)
  
  DFM_model_modify = dfm.DeepFactorizationMachineModel([100 for i in range(num_field)],dim,l,0.1)
  modify_traced = symbolic_trace(DFM_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_SingleMLP(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [6]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [7]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_dfm,l = [1024,512,256]):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    for i in range(100):
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)  
        start_event.record()

        with torch.no_grad():
          soutput = compiled_model(t)
        end_event.record()
        torch.cuda.synchronize()
        # 计算并打印函数执行所需的时间
        elapsed_time = start_event.elapsed_time(end_event)
        total_time.append(elapsed_time)
    print(calculate_mean_and_variance_manual(total_time[2:]))
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,l)
  run(ori)
  run(modify)
  torch.cuda.empty_cache()

In [8]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :1024
now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 421 / 1024 (41.1%)
Greatest absolute difference: 0.003106832504272461 at index (335,) (up to 1e-05 allowed)
Greatest relative difference: 0.016433497192990774 at index (838,) (up to 1e-05 allowed)
  _check_trace(


(0.7678625291707565, 0.00015225750135138214)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 387 / 1024 (37.8%)
Greatest absolute difference: 0.003786921501159668 at index (477,) (up to 1e-05 allowed)
Greatest relative difference: 0.019578785224598636 at index (443,) (up to 1e-05 allowed)
  _check_trace(


(0.610463346145591, 2.9647356707999983e-05)


In [9]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 765 / 2048 (37.4%)
Greatest absolute difference: 0.0035720467567443848 at index (348,) (up to 1e-05 allowed)
Greatest relative difference: 0.018446238380668285 at index (1455,) (up to 1e-05 allowed)
  _check_trace(


(1.4517756724844173, 0.0005245439653062225)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 788 / 2048 (38.5%)
Greatest absolute difference: 0.0031389594078063965 at index (1203,) (up to 1e-05 allowed)
Greatest relative difference: 0.014102082569765648 at index (330,) (up to 1e-05 allowed)
  _check_trace(


(1.1062644866048073, 0.00028658187401352914)


In [10]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1512 / 4096 (36.9%)
Greatest absolute difference: 0.004056960344314575 at index (263,) (up to 1e-05 allowed)
Greatest relative difference: 0.018764824656426468 at index (2732,) (up to 1e-05 allowed)
  _check_trace(


(2.647144149760811, 0.003317753081547872)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1544 / 4096 (37.7%)
Greatest absolute difference: 0.003874868154525757 at index (448,) (up to 1e-05 allowed)
Greatest relative difference: 0.019614351796630493 at index (1227,) (up to 1e-05 allowed)
  _check_trace(


(2.0051301219025435, 0.02067186983466148)


In [11]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :8192
now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3092 / 8192 (37.7%)
Greatest absolute difference: 0.004181325435638428 at index (6833,) (up to 1e-05 allowed)
Greatest relative difference: 0.020852688175572905 at index (7589,) (up to 1e-05 allowed)
  _check_trace(


(5.1547876036897, 0.006501965622148635)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3039 / 8192 (37.1%)
Greatest absolute difference: 0.004620373249053955 at index (1138,) (up to 1e-05 allowed)
Greatest relative difference: 0.02051884143199956 at index (1142,) (up to 1e-05 allowed)
  _check_trace(


(3.7887771470206126, 0.0021589956359975427)


In [12]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :1024
now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 381 / 1024 (37.2%)
Greatest absolute difference: 0.0028774142265319824 at index (436,) (up to 1e-05 allowed)
Greatest relative difference: 0.013571264771741585 at index (224,) (up to 1e-05 allowed)
  _check_trace(


(1.2092208898797328, 0.001038026348683049)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 347 / 1024 (33.9%)
Greatest absolute difference: 0.0029754042625427246 at index (1016,) (up to 1e-05 allowed)
Greatest relative difference: 0.014114035932911466 at index (398,) (up to 1e-05 allowed)
  _check_trace(


(0.9313253857651536, 0.00024092040424619762)


In [13]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :2048
now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 799 / 2048 (39.0%)
Greatest absolute difference: 0.0037912726402282715 at index (472,) (up to 1e-05 allowed)
Greatest relative difference: 0.016118882804358452 at index (1254,) (up to 1e-05 allowed)
  _check_trace(


(2.449498772621155, 0.0027765470593666395)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 773 / 2048 (37.7%)
Greatest absolute difference: 0.003050088882446289 at index (116,) (up to 1e-05 allowed)
Greatest relative difference: 0.014764956799313363 at index (1089,) (up to 1e-05 allowed)
  _check_trace(


(1.785202610249422, 0.0005516651626921611)


In [14]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :4096
now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1620 / 4096 (39.6%)
Greatest absolute difference: 0.00448259711265564 at index (2054,) (up to 1e-05 allowed)
Greatest relative difference: 0.015790717162102716 at index (2662,) (up to 1e-05 allowed)
  _check_trace(


(4.638654703996619, 0.00818072594875834)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1577 / 4096 (38.5%)
Greatest absolute difference: 0.004310339689254761 at index (1175,) (up to 1e-05 allowed)
Greatest relative difference: 0.023435471786665846 at index (260,) (up to 1e-05 allowed)
  _check_trace(


(3.2604515795805016, 0.0015672466276737517)


In [15]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :8192
now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3035 / 8192 (37.0%)
Greatest absolute difference: 0.003476738929748535 at index (5065,) (up to 1e-05 allowed)
Greatest relative difference: 0.012885847745560808 at index (2064,) (up to 1e-05 allowed)
  _check_trace(


(9.374508069486033, 0.0282986918605001)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3209 / 8192 (39.2%)
Greatest absolute difference: 0.005105108022689819 at index (2487,) (up to 1e-05 allowed)
Greatest relative difference: 0.02276023503371717 at index (824,) (up to 1e-05 allowed)
  _check_trace(


(6.368749409305806, 0.007170825833624686)


In [16]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 334 / 1024 (32.6%)
Greatest absolute difference: 0.003099501132965088 at index (567,) (up to 1e-05 allowed)
Greatest relative difference: 0.015344604890485376 at index (514,) (up to 1e-05 allowed)
  _check_trace(


(1.0038233502786986, 0.0008848665385308893)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 323 / 1024 (31.5%)
Greatest absolute difference: 0.003623664379119873 at index (900,) (up to 1e-05 allowed)
Greatest relative difference: 0.01542563248627089 at index (664,) (up to 1e-05 allowed)
  _check_trace(


(0.5776408168734336, 5.3780133284882903e-05)


In [17]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 623 / 2048 (30.4%)
Greatest absolute difference: 0.0038920044898986816 at index (837,) (up to 1e-05 allowed)
Greatest relative difference: 0.015291849701250502 at index (1886,) (up to 1e-05 allowed)
  _check_trace(


(1.9561152628489904, 0.0017714572087618424)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 668 / 2048 (32.6%)
Greatest absolute difference: 0.003909945487976074 at index (1926,) (up to 1e-05 allowed)
Greatest relative difference: 0.014693824582008592 at index (1404,) (up to 1e-05 allowed)
  _check_trace(


(1.0236888187272208, 0.0004795772371755873)


In [18]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1214 / 4096 (29.6%)
Greatest absolute difference: 0.0028392672538757324 at index (1009,) (up to 1e-05 allowed)
Greatest relative difference: 0.014726150593605368 at index (2950,) (up to 1e-05 allowed)
  _check_trace(


(3.7627882349247836, 0.005347876208710958)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1241 / 4096 (30.3%)
Greatest absolute difference: 0.0034744739532470703 at index (4075,) (up to 1e-05 allowed)
Greatest relative difference: 0.019958823061817035 at index (3276,) (up to 1e-05 allowed)
  _check_trace(


(1.7977923239980425, 0.0003114031920053383)


In [19]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :8192
now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2588 / 8192 (31.6%)
Greatest absolute difference: 0.003994375467300415 at index (844,) (up to 1e-05 allowed)
Greatest relative difference: 0.018415775590746104 at index (2928,) (up to 1e-05 allowed)
  _check_trace(


(7.551220582456005, 0.009071510638390082)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2356 / 8192 (28.8%)
Greatest absolute difference: 0.0029758214950561523 at index (7036,) (up to 1e-05 allowed)
Greatest relative difference: 0.014591001035315972 at index (4823,) (up to 1e-05 allowed)
  _check_trace(


(3.4320382050105502, 0.000592730406622752)


In [20]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :1024
now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 293 / 1024 (28.6%)
Greatest absolute difference: 0.0028390884399414062 at index (462,) (up to 1e-05 allowed)
Greatest relative difference: 0.014715959080732907 at index (490,) (up to 1e-05 allowed)
  _check_trace(


(1.7653508612087794, 0.0027566068087989597)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 331 / 1024 (32.3%)
Greatest absolute difference: 0.004569947719573975 at index (812,) (up to 1e-05 allowed)
Greatest relative difference: 0.01627532200805101 at index (679,) (up to 1e-05 allowed)
  _check_trace(


(0.8776107746727613, 8.177777799827755e-05)


In [21]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :2048
now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 622 / 2048 (30.4%)
Greatest absolute difference: 0.0041351318359375 at index (1329,) (up to 1e-05 allowed)
Greatest relative difference: 0.014047441696370637 at index (518,) (up to 1e-05 allowed)
  _check_trace(


(3.5529547710807954, 0.006140963875011588)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 635 / 2048 (31.0%)
Greatest absolute difference: 0.004169255495071411 at index (1559,) (up to 1e-05 allowed)
Greatest relative difference: 0.016526383349939718 at index (1684,) (up to 1e-05 allowed)
  _check_trace(


(1.6036022877206608, 0.00015789436649062415)


In [22]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1303 / 4096 (31.8%)
Greatest absolute difference: 0.00344809889793396 at index (2723,) (up to 1e-05 allowed)
Greatest relative difference: 0.01639330779794546 at index (1131,) (up to 1e-05 allowed)
  _check_trace(


(6.944070859831207, 0.007857507453277072)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1245 / 4096 (30.4%)
Greatest absolute difference: 0.003508925437927246 at index (2537,) (up to 1e-05 allowed)
Greatest relative difference: 0.017471905235140868 at index (2017,) (up to 1e-05 allowed)
  _check_trace(


(2.9163859596057815, 0.00040535269408352627)
